# Killer AI Minimap creation

This notebook helps show how a minimap is made from a given video. For all detected objects, they will appear as certain symbols within a minimap.

## 1. Imports and variable setup

In [1]:
# Import all libraries
import os
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image, Video
from pathlib import Path
import sys
import glob
import random
import cv2
import numpy as np
import supervision as sv
# from dbdkillerai.models.predict_model import process_frame
# Setup HOME environment variable
HOME = os.getcwd()
print(HOME)
from ipywidgets import Video as ipyVideo

/Users/mreagles524/anaconda3/envs/dbd_ai/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mreagles524/anaconda3/envs/dbd_ai/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/mreagles524/anaconda3/envs/dbd_ai/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <F9F049DF-FB3A-3319-9850-7654881F4195> /Users/mreagles524/anaconda3/envs/dbd_ai/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


/Users/mreagles524/Documents/gitrepos/projects/DBD-Killer-AI/notebooks


In [2]:
# Check image displays are good
# display.clear_output()
sys.path.append(str(Path.cwd().parent))
# Check ultralytics library is good
ultralytics.checks()

Ultralytics YOLOv8.0.238 🚀 Python-3.10.13 torch-2.2.0 CPU (Apple M1 Pro)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 96.4/460.4 GB disk)


In [3]:
# Environment variables
PROJECT_DIR = Path.cwd().parent
DATASET_VERSION = 5

In [4]:
data_directory = str(PROJECT_DIR) + f"/data/external/deadbydaylightkillerai/killer_ai_object_detection/{DATASET_VERSION}"
yaml_directory = data_directory + "/data.yaml"

## 2. Import Model

Find the best model amongst all runs of training runs.

**Note**:
- mAP under "train" folders correspond to mAP agaisnt validation data
- mAP under "val" folders correspond to map against test data
    - Per this response from developers: https://github.com/ultralytics/ultralytics/issues/3737#issuecomment-1813520866

To move along in the project, we will only focus on models that ran against validation data.

In [5]:
from dbdkillerai.utils.utils import find_best_model

# Grab the best model from the runs directory, only validation(train) runs
# target_data_value is the desired dataset to filter runs from.
best_model_info = find_best_model(runs_directory="runs/detect/",
                                   folder_type="train", # train = "val" mAP
                                   target_data_value=f"deadbydaylightkillerai/killer_ai_object_detection/{DATASET_VERSION}")
if best_model_info is not None:
    run_folder = best_model_info["model_folder"]
    model = YOLO(model=best_model_info["weights_path"])

No models trained on dataset 5


## 3. Process video

A video file will pass through a given video to create a close-up minimap.

In [ ]:
# Open the video file
VIDEO_PATH = f"{PROJECT_DIR}/data/raw/huntress_chase.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
